# Creating first agent

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import the libraries

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [3]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the agent

In [4]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_zBCZWsxukSKl3IwvFNTxHWPL


## Create thread

In [5]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_gF2x6jIGredu4l99FMcmcyc8


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [ ]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [7]:
user_input = "What are the 5 things I can see in New York?"
run_agent(user_input)

Created message, ID: msg_CRhC7GKshS5VFQfXF6AzV19h
Run finished with status: completed
Last Message: New York has a plethora of iconic landmarks and attractions to offer. Here are five notable things you can see in New York:

1. **Statue of Liberty and Ellis Island**: The Statue of Liberty is an enduring symbol of freedom and democracy, and Ellis Island is a historic site where millions of immigrants passed through when coming to America.

2. **Central Park**: This vast green oasis in the middle of Manhattan offers walking paths, lakes, a zoo, and numerous recreational opportunities.

3. **Times Square**: Known as "The Crossroads of the World," Times Square is famous for its bright, neon lights, Broadway theaters, and bustling atmosphere.

4. **Empire State Building**: One of the most famous skyscrapers in the world, offering stunning panoramic views of the city from its observation decks.

5. **Metropolitan Museum of Art (the Met)**: One of the largest and most prestigious art museums 

## Cleanup resources

In [9]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent


## Creating an agent and thread

In [10]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )
print(f"Created agent, ID: {agent.id}")
agent_id = agent.id


Created agent, ID: asst_rYtqThaxtV1kS4CQrs4zedMm


## Retrieving an agent

In [11]:
agent = project_client.agents.get_agent(agent_id)

## Create thread

To retrieve a thread
- thread = project.agents.threads.get("")

In [12]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_iNSMJe0ic66akMVWxdlw27U5


## Run agent

In [13]:
user_input = "What are the 5 things I can see in San Francisco?"
run_agent(user_input)

Created message, ID: msg_9R0bbk9bPucq7tbZAwvhN2zU
Run finished with status: completed
Agent Response: San Francisco is a vibrant city with many iconic landmarks and attractions. Here are five must-see sights:

1. **Golden Gate Bridge**: One of the most famous bridges in the world, known for its stunning architecture and breathtaking views. You can walk, bike, or drive across it.

2. **Alcatraz Island**: This former federal prison, located on an island in San Francisco Bay, once housed notorious criminals like Al Capone. You can take a ferry to the island and explore the prison on a guided tour.

3. **Fisherman's Wharf**: A bustling waterfront area that offers a variety of seafood restaurants, shops, and attractions. Don't miss the sea lions at Pier 39 and the historic ships at Hyde Street Pier.

4. **Chinatown**: San Francisco's Chinatown is the largest outside of Asia and the oldest in North America. Wander through its vibrant streets, visit traditional shops, and enjoy delicious Chin

## Cleanup resources

In [14]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
